## Load Libs

In [ ]:
%reload_ext autoreload
%autoreload 2

!pip install seaborn --upgrade

import os, sys, numpy as np, pandas as pd, tensorflow as tf, cv2
import seaborn as sns
import scipy.stats as stats

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, LabelBinarizer
from sklearn.metrics import auc, roc_curve, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from matplotlib import pyplot as plt
sns.set(style="white")

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, BatchNormalization, Activation, Input, Dropout, Embedding, Flatten, Input
from keras.layers import Concatenate
from keras.optimizers import Adam, Nadam
from keras import regularizers

%matplotlib inline

def plot_result(hist):
    plt.figure(figsize=(16, 5))

    plt.subplot(1, 2, 1)
    plt.plot(hist.history['loss'], label='tr_loss')
    plt.plot(hist.history['val_loss'], label='vl_loss')
    plt.title('Loss')

    plt.subplot(1, 2, 2)
    plt.plot(hist.history['acc'], label='acc')
    plt.plot(hist.history['val_acc'], label='val_acc')
    plt.title('Accuracy')

    plt.legend(loc='best')
    plt.show()
    
def draw_roc_curve(y, pred):
    from matplotlib import pyplot as plt

    fprRf, tprRf, _ = roc_curve(y, pred, pos_label=1)
    auc_scr = auc(fprRf, tprRf)
    print("auc:", auc_scr)
    f, ax = plt.subplots(1, 1, figsize=(6, 6))

    ax.plot([0, 1], [0, 1], 'k--')
    ax.plot(fprRf, tprRf, label='ROC CURVE')
    ax.set_xlabel('False positive rate')
    ax.set_ylabel('True positive rate')
    ax.set_title('Area Under Curve(ROC) (score: {:.4f})'.format(auc_scr))
    ax.legend(loc='best')
    plt.show()

## Data Schema

Column name     | Description
             ---|---
RorNumber       |    Row number

---


---



# Purpose: 
* 

In [ ]:
# raw = pd.read_csv("https://storage.googleapis.com/allianz-course/data/raw_tr_financial_churn.csv")
# raw_vl = pd.read_csv("https://storage.googleapis.com/allianz-course/data/raw_vl_financial_churn.csv")
# raw.head()
raw = pd.read_csv('../data/raw_financial_distess.csv').rename(columns={'Financial Distress': 'distress_num'})
raw.head(10)

In [ ]:
raw.info()

## Features setting

In [ ]:
catg_features = ['x80']
num_features = [f'x{i}' for i in range(1, 84) if i != 80]
# embedding_features = catg_features + ['binn_tenure', 'binn_MonthlyCharges', 'binn_TotalCharges']
label = 'distress_num'

<br/>

---



In [ ]:
# Rolling mean
raw['rolling_mean'] = (raw.groupby('Company', group_keys=False)
                          .apply(lambda pipe: pipe['distress_num'].rolling(window=3).mean().fillna(0))
                          .values)

In [ ]:
# Split train valid
def split_fn(pipe):
    lens = len(pipe)
    ret = np.ones(lens)
    if lens >= 5:
        split_size = int(lens * 0.36)
        ret[-split_size:] = 0
    return ret

raw['is_train'] = np.concatenate(raw.groupby('Company').apply(split_fn).values)
raw_vl = raw.query("is_train == 0").drop('is_train', 1)
raw = raw.query("is_train == 1").drop('is_train', 1)
raw.head()

In [ ]:
corr = raw.corr()
corr['distress_num'][(corr['distress_num'] <= -0.3) | (corr['distress_num'] >= 0.3)]

## Drop `Company`, Add `distress_catg`

In [ ]:
# Categorical label, not for training of course, for target mean encoding
raw['distress_catg'] = (raw.distress_num <= 0.5).astype(int)
# 
raw.drop(['Company'], 1, inplace=True)

## Base Feature Preprocessing

* Normalize ofr numeric features
* One Hot Encoding for categorical features

In [ ]:
def feature_engineering(raw, is_train=True, status=None):
    data = raw.copy()
    
    if is_train:
        status = {'scaler': None, 
                  'mapper': defaultdict(LabelBinarizer),
                  'woe_mapper': {},
                  'binn_mapper': {},
                  'freq_mapper': {},
                  'mean_mapper': {}}

    catg_ftrs = catg_features.copy()
        
    # Numeric binning to categorical features
    do_binning(catg_ftrs, status, data, is_train)
    # -----------------------------------------------------------------------------------------

    catg_part = pd.DataFrame()
    
    # One Hot Encoding
    # catg_part = pd.get_dummies(data[catg_ftrs])
    # do_onehot(catg_ftrs, catg_part, status, data, is_train)
    # -----------------------------------------------------------------------------------------

    # Embedding, simple indexing categorical features, actual embedding function in keras
    do_embedding(catg_ftrs, catg_part, status, data, is_train)
    # -----------------------------------------------------------------------------------------

    # WOE encoding
    do_woe_encoding(catg_ftrs, catg_part, status, data, is_train)
    # -----------------------------------------------------------------------------------------

    # Frequency encoding, target mean encoding
    do_target_encoding(catg_ftrs, catg_part, status, data, is_train)
    # -----------------------------------------------------------------------------------------

    # Normalize
    num_part = do_norm(num_features, status, data, is_train)
    # -----------------------------------------------------------------------------------------
    
    # Merge categorical, numeric features
    data_x, data_y = pd.concat([catg_part, num_part], 1), data['distress_num'].copy()
    
    # Include quadratic term, cube term
    do_nth_order_polynominal(num_features, data_x)
    # ----------------------------------------------------------------------------------------- 
    return data_x, data_y, status


## Functions of Features Processing

In [ ]:
def do_binning(catg_ftrs, status, data, is_train):
    pass

def do_onehot(catg_ftrs, catg_part, status, data, is_train):
    mapper = status['mapper']
    tmp = []
    for catg_col in catg_ftrs:
        if is_train:
            result = mapper[catg_col].fit_transform(data[catg_col])
        else:
            result = mapper[catg_col].transform(data[catg_col])

        columns = [f'{catg_col}_{col}' for col in mapper[catg_col].classes_]
        if result.shape[1] == 1:
            columns = columns[:1]
        tmp.append(pd.DataFrame(data=result, columns=columns))
    tmp = pd.concat(tmp, 1)
    for col in tmp:
        catg_part[col] = tmp[col]
    pass

def do_woe_encoding(catg_ftrs, catg_part, status, data, is_train):
    pass

def do_target_encoding(catg_ftrs, catg_part, status, data, is_train):
    pass

def do_norm(num_features, status, data, is_train):
    num_part = data[num_features].copy()
    if is_train:
        scaler = StandardScaler()
        status['scaler'] = scaler
        num_part = pd.DataFrame(data=scaler.fit_transform(num_part), columns=num_part.columns)
    else:
        scaler = status['scaler']
        num_part = pd.DataFrame(data=scaler.transform(num_part), columns=num_part.columns)
    return num_part
    pass

def do_nth_order_polynominal(num_features, data):
    pass

In [ ]:
# Trnasform raw data to foramt model recognized
tr_x, tr_y, status = feature_engineering(raw, is_train=True)
vl_x, vl_y, _ = feature_engineering(raw_vl, is_train=False, status=status)

print( tr_x.shape, vl_x.shape )
tr_x.head()

## Model on Base Featue Engineering

* Network Structure: 
    * `dimension of input, 64, 32, 16, 1`
* ReLU Activation Function
* Adam Optimizer
* Sigmoid Cross Entropy

In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
K.clear_session()

model = get_model(input_dim=tr_x.shape[1])
hist = model.fit(tr_x, tr_y, validation_data=(vl_x, vl_y), batch_size=100, epochs=30)
plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x))

## Analyze the Prediction

In [ ]:
y_pred = model.predict(vl_x)
y_pred

In [ ]:
def pred_dist(y_pred):
    y_pred = pd.Series(y_pred.ravel())
    print( y_pred.describe() )
    print( y_pred.mean(), y_pred.std() )
    sns.lineplot(range(len(y_pred)), sorted(y_pred.ravel()))
    plt.show()

    sns.boxplot(y_pred.ravel(), orient='v')
    plt.show()

    sns.distplot(y_pred.ravel())
    plt.show()
    
pred_dist(y_pred)

## Observation

* 預測機率範圍集中在 `0.273`左右, 已經是小數點3位以下的差距
* Accuracy預設threshold = 0.5, model不會產出Positive的預測
* 問題變成找出最佳Threshold ==> 預測機率大於多少時認為是`Exited = 1`

---


## 找出最佳閥值

In [ ]:
# 依照所有的閥值(切割100等分)算出F score, 找出分數最高的閥值
def f_beta_scann(y_true, y_pred, beta=0.5):
    y_pred = pd.Series(y_pred.ravel())
    # 切割100等分, 尋找最佳 f beta score
    bins = np.linspace(y_pred.min(), y_pred.max(), 100)
    # 找出F beta score最高的點
    result = np.array([precision_recall_fscore_support(y_true=y_true, y_pred=y_pred > thres, beta=beta)[2][1] 
                       for thres in bins])
    best_idx = result.argmax()
    return bins[best_idx], result[best_idx]

def analyze(y_true, y_pred):
    thres, f_beta = f_beta_scann(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred.ravel() > thres)
    print(f'thres: {thres:.4f}, f_beta score: {f_beta:.4f}')
    print(f'accuracy@{thres:.4f}: {acc:.2f}\n')

    print(f'threshold@0.5, accuracy: {accuracy_score(y_true, y_pred.ravel() > 0.5):.4f}')
    print(confusion_matrix(y_true, y_pred.ravel() > 0.5), '\n')

    print(f'threshold@{thres:.4f}, accuracy: {accuracy_score(y_true, y_pred.ravel() > thres):.4f}')
    print(confusion_matrix(y_true, y_pred.ravel() > thres))
    
analyze(vl_y, y_pred)

## Observation

* 整體Accuracy 0.75, 卻無法產出 `Exited = 1` 的預測
* 經果F beta score掃描之後, 整體Accuracy 降低, 但能夠產出退租名單
* 根據以上, 是否還會糾結於 Accuracy 的高低?

## Bining Numeric Feature

* Quartile cut
* `Age`欄位特殊處理 => [14, 30, 35, 40, 45, 62]
* `Balance`欄位特殊處理 => `IsBalanceZero`


In [ ]:
def do_binning(catg_ftrs, status, data, is_train):
    # Quartile binning, outlier set to class "0"
    def quartile_binning(x):
        bins = np.percentile(x, range(0, 100, 25))[1:].tolist()
        iqr_x_150 = (bins[-1] - bins[0]) * 1.5
        bins = [bins[0] - iqr_x_150] + bins + [bins[-1] + iqr_x_150]
        result = pd.Series(np.digitize(x, bins)).map(pd.Series([0, 1, 2, 3, 4, 0])).values
        return result, bins
    
    # Age
    bins = np.array([14, 30, 35, 40, 45, 62])
    labels = ['~14', '14-30', '30-35', '35-40', '40-45', '45-62', '62up']
    age_map = pd.Series(labels)
    data['binn_Age'] = pd.Series(np.digitize(data.Age, bins)).map(age_map).values
    catg_ftrs.append('binn_Age')
    # Balance
    data['binn_Balance'] = data.Balance.map(lambda e: 0 if e == 0 else 1)
    catg_ftrs.append('binn_Balance')
    
    for col in ('CreditScore', 'Tenure', 'EstimatedSalary'):
        binned_name = f'binn_{col}'
        if is_train:
            result, bins = quartile_binning(data[col])
            status['binn_mapper'][binned_name] = bins
            data[binned_name] = result
        else:
            bins = status['binn_mapper'][binned_name]
            data[binned_name] = pd.Series(np.digitize(data[col], bins))\
                                  .map(pd.Series([0, 1, 2, 3, 4, 0])).values

        catg_ftrs.append(binned_name)
    pass

In [ ]:
# Trnasform raw data to foramt model recognized
tr_x, tr_y, status = feature_engineering(raw, is_train=True)
vl_x, vl_y, _ = feature_engineering(raw_vl, is_train=False, status=status)

print( tr_x.shape, vl_x.shape )
tr_x.head()

## Model (Binning)

In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 16, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 16, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

K.clear_session()

model = get_model(input_dim=tr_x.shape[1])
hist = model.fit(tr_x, tr_y, validation_data=(vl_x, vl_y), batch_size=100, epochs=30)
plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x))

y_pred = model.predict(vl_x)
pred_dist(y_pred)
analyze(vl_y, y_pred)

## Add Weight of Evidence Encoding

In [ ]:
def do_woe_encoding(catg_ftrs, catg_part, status, data, is_train):

    def woe_encode(x, label, data):
        """Calculate the Weight of Evidence of given categorical feature and label

        :param x: Given feature name
        :param label: Label name
        :param data:
        :return: WOE encoded dictionary
        """
        total_vc = data[label].value_counts().sort_index()

        def woe(pipe, total_vc):
            # Count by label in this group
            group_vc = pipe[label].value_counts().sort_index()

            # Some class in the feature is missing, fill zero to missing class
            if len(group_vc) < len(total_vc):
                for key in total_vc.index:
                    if key not in group_vc:
                        group_vc[key] = 0.
                group_vc = group_vc.sort_index()

            # WOE formula
            r = ((group_vc + 0.5) / total_vc).values

            # Odd ratio => 1 to 0, you can define meaning of each class
            return np.log(r[1] / r[0])

        return data.groupby(x).apply(lambda pipe: woe(pipe, total_vc))

    for catg_col in catg_ftrs:
        if is_train:
            kv = woe_encode(catg_col, 'Exited', data)
            status['woe_mapper'][catg_col] = kv.to_dict()
        else:
            kv = pd.Series(status['woe_mapper'][catg_col])
        catg_part[f'woe_{catg_col}'] = kv.reindex(data[catg_col]).values
    pass

In [ ]:
# Trnasform raw data to foramt model recognized
tr_x, tr_y, status = feature_engineering(raw, is_train=True)
vl_x, vl_y, _ = feature_engineering(raw_vl, is_train=False, status=status)

print( tr_x.shape, vl_x.shape )
tr_x.head()

## Model (WOE)

In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 20, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 20, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

K.clear_session()

model = get_model(input_dim=tr_x.shape[1])
hist = model.fit(tr_x, tr_y, validation_data=(vl_x, vl_y), batch_size=100, epochs=30)
plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x))

y_pred = model.predict(vl_x)
pred_dist(y_pred)
analyze(vl_y, y_pred)

## Statistical Encoding, Target Encoding

* Frequency encoding
* Mean encoding

In [ ]:
def do_target_encoding(catg_ftrs, catg_part, status, data, is_train):
    for catg_col in catg_ftrs:
        if is_train:
            freq_proportion = data[catg_col].value_counts() / len(data)
            catg_part[f'freq_{catg_col}'] = freq_proportion.reindex(data[catg_col]).values
            target_mean = data.groupby(catg_col).Exited.mean()
            catg_part[f'mean_{catg_col}'] = target_mean.reindex(data[catg_col]).values

            status['freq_mapper'][catg_col] = freq_proportion.to_dict()
            status['mean_mapper'][catg_col] = target_mean.to_dict()
        else:
            catg_part[f'freq_{catg_col}'] = pd.Series(status['freq_mapper'][catg_col]).reindex(data[catg_col]).values
            catg_part[f'mean_{catg_col}'] = pd.Series(status['mean_mapper'][catg_col]).reindex(data[catg_col]).values
    pass

In [ ]:
# Trnasform raw data to foramt model recognized
tr_x, tr_y, status = feature_engineering(raw, is_train=True)
vl_x, vl_y, _ = feature_engineering(raw_vl, is_train=False, status=status)

print( tr_x.shape, vl_x.shape )
tr_x.head()

## Model (WOE + Binning + Freqency Encoding + Target Mean Encoding)

In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 30, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 30, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

K.clear_session()

model = get_model(input_dim=tr_x.shape[1])
hist = model.fit(tr_x, tr_y, validation_data=(vl_x, vl_y), batch_size=100, epochs=30)
plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x))

y_pred = model.predict(vl_x)
pred_dist(y_pred)
analyze(vl_y, y_pred)

## Add Polynomial Featue Engineering

* Add quradratic, cube term base on numeric features

In [ ]:
def do_nth_order_polynominal(num_features, data):
    for num_col in num_features:
        data[f'{num_col}_degree_2'] = data[num_col] ** 2
        data[f'{num_col}_degree_3'] = data[num_col] ** 3
    pass

In [ ]:
# Trnasform raw data to foramt model recognized
tr_x, tr_y, status = feature_engineering(raw, is_train=True)
vl_x, vl_y, _ = feature_engineering(raw_vl, is_train=False, status=status)

print( tr_x.shape, vl_x.shape )
tr_x.head()

## Model (WOE + Binning + Freqency Encoding + Target Mean Encoding + Polynominal)


In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 35, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 35, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

K.clear_session()

model = get_model(input_dim=tr_x.shape[1])
hist = model.fit(tr_x, tr_y, validation_data=(vl_x, vl_y), batch_size=100, epochs=30)
plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x))

y_pred = model.predict(vl_x)
pred_dist(y_pred)
analyze(vl_y, y_pred)

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=64)
tr_x_pca = pca.fit_transform(tr_x)
vl_x_pca = pca.transform(vl_x)

In [ ]:
tr_x_pca.shape, vl_x_pca.shape

In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 32, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 32, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

K.clear_session()

model = get_model(input_dim=tr_x_pca.shape[1])
hist = model.fit(tr_x_pca, tr_y, validation_data=(vl_x_pca, vl_y), batch_size=100, epochs=30)
plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x_pca))

y_pred = model.predict(vl_x_pca)
pred_dist(y_pred)
analyze(vl_y, y_pred)

## Model (AutoEncoder)

In [ ]:
# Trnasform raw data to foramt model recognized
tr_x, tr_y, status = feature_engineering(raw, is_train=True)
vl_x, vl_y, _ = feature_engineering(raw_vl, is_train=False, status=status)

print( tr_x.shape, vl_x.shape )
tr_x.head()

In [ ]:
K.clear_session()

inputs_dim = tr_x.shape[1]

inputs = Input(shape=(inputs_dim, ))
# Encoder
encoded = Dense(inputs_dim, activation='selu')(inputs)
encoded = Dense(128, activation='selu')(encoded)
encoded = Dense(64, activation='selu')(encoded)
encoded = Dense(64, activation='selu')(encoded)

# Decoder
decoded = Dense(64, activation='selu')(encoded)
decoded = Dense(64, activation='selu')(decoded)
decoded = Dense(128, activation='selu')(decoded)
decoded = Dense(inputs_dim, activation='linear')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(inputs, decoded)

# this model maps an input to its encoded representation
encoder = Model(inputs, encoded)

# Adam Optimizer + Mean square error loss
autoencoder.compile(optimizer='adam', loss='mse')
hist = autoencoder.fit(
    tr_x, tr_x, 
    epochs=100, 
    batch_size=200, 
    shuffle=True, 
    validation_data=(vl_x, vl_x))

def plot_ae(hist):
    plt.figure(figsize=(16, 5))
    plt.plot(hist.history['loss'], label='tr_loss')
    plt.plot(hist.history['val_loss'], label='vl_loss')
    plt.title('Loss')
    plt.legend(loc='best')
    plt.show()
    
plot_ae(hist)

In [ ]:
tr_x_ae = encoder.predict(tr_x)
vl_x_ae = encoder.predict(vl_x)

In [ ]:
tr_x_ae.shape, vl_x_ae.shape

In [ ]:
def get_model(input_dim):
    model = Sequential()
    # kernel_regularizer=regularizers.l2(0.005)
    model.add(Dense(units = 128, activation = 'relu', kernel_initializer='uniform', input_dim = input_dim))
    model.add(Dense(units = 128, activation = 'relu', kernel_initializer='uniform'))
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

K.clear_session()

model = get_model(input_dim=tr_x_ae.shape[1])
hist = model.fit(tr_x_ae, tr_y, validation_data=(vl_x_ae, vl_y), batch_size=100, epochs=30)

plot_result(hist)
draw_roc_curve(vl_y, model.predict(vl_x_ae))

y_pred = model.predict(vl_x_ae)
pred_dist(y_pred)
analyze(vl_y, y_pred)